In [15]:
import import_ipynb
from mydata import getHotpotData
from mydata import dataProcesser
import argparse
import spacy
import csv
import pyprind
from torchtext import data
from torchtext.vocab import GloVe
import numpy as np
import torch
from torchtext.data import Iterator, BucketIterator
from spacy.symbols import ORTH
from QueryEncoder import CSAEncoder

parser = argparse.ArgumentParser()
parser.add_argument('--batch-size', default=32, type=int)
parser.add_argument('--gpu', default=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'), type=int)
parser.add_argument('--csa-mode',default='add',type = str)
parser.add_argument('--word-dim',default=300,type = int)
parser.add_argument('--block-size', default=-1, type=int)
parser.add_argument('--mSA-scalar', default=5.0, type=float)
args = parser.parse_args(args=[])

trainpath = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/small_train_sep_100.csv'
devpath = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/small_dev_sep_100.csv'



In [16]:
mydata = getHotpotData(args,trainpath,devpath)
print('done')

load hotpot data done
done


In [24]:
def spilter(x,tk):
    res = []
    s = 0
    sqlen = len(x)
    for i,t in enumerate(x): 
        if t == tk:
            res.append(x[s:i])
            s = i+1   
    if s is 0:       
        res.append(x[s:-1])
    return res

for i,ex in enumerate(mydata.train.examples):
    #print(spilter(mydata.train.examples[i].Answer,'<sep>'))
    print(mydata.train.examples[i].Answer)
    print(spilter(mydata.train.examples[i].Answer,'<sep>'))
    print('========\n')
    #mydata.train.examples[i].Question = spilter(mydata.train.examples[i].Question,'<sep>')
    #mydata.train.examples[i].Passage = spilter(mydata.train.examples[i].Passage,'<sep>')
    #print(mydata.train.examples[i].Passage)
#print(spilter(mydata.train.examples[0].Passage,'<sep>'))

['arthur', "'s", 'magazine', '(', '1844–1846', ')', 'was', 'an', 'american', 'literary', 'periodical', 'published', 'in', 'philadelphia', 'in', 'the', '19th', 'century', '.', '<sep>', 'first', 'for', 'women', 'is', 'a', 'woman', "'s", 'magazine', 'published', 'by', 'bauer', 'media', 'group', 'in', 'the', 'usa', '.', '<sep>']
[['arthur', "'s", 'magazine', '(', '1844–1846', ')', 'was', 'an', 'american', 'literary', 'periodical', 'published', 'in', 'philadelphia', 'in', 'the', '19th', 'century', '.'], ['first', 'for', 'women', 'is', 'a', 'woman', "'s", 'magazine', 'published', 'by', 'bauer', 'media', 'group', 'in', 'the', 'usa', '.']]

['the', 'oberoi', 'family', 'is', 'an', 'indian', 'family', 'that', 'is', 'famous', 'for', 'its', 'involvement', 'in', 'hotels', ',', 'namely', 'through', 'the', 'oberoi', 'group', '.', '<sep>', 'the', 'oberoi', 'group', 'is', 'a', 'hotel', 'company', 'with', 'its', 'head', 'office', 'in', 'delhi', '.', '<sep>']
[['the', 'oberoi', 'family', 'is', 'an', 'ind

[' ', 'fighters', 'from', 'around', 'world', 'on', 'the', 'roster', 'include', 'badr', 'hari', ',', 'peter', 'aerts', ',', 'peter', 'graham', ',', 'dewey', 'cooper', ',', 'zabit', 'samedov', '.', '<sep>', ' ', 'it', 'was', 'considered', 'as', 'one', 'of', 'the', 'biggest', 'kickboxing', 'and', 'mma', 'promotion', 'in', 'middle', 'east', '.', '<sep>', 'badr', 'hari', '(', 'arabic', ':', 'بدر', 'هاري\u200e', '\u200e', ';', 'born', '8', 'december', '1984', ')', 'is', 'a', 'moroccan', '-', 'dutch', 'super', 'heavyweight', 'kickboxer', 'from', 'amsterdam', ',', 'fighting', 'out', 'of', 'mike', "'s", 'gym', 'in', 'oostzaan', '.', '<sep>', ' ', 'hari', 'has', 'been', 'a', 'prominent', 'figure', 'in', 'the', 'world', 'of', 'kickboxing', 'and', 'was', 'once', 'considered', 'the', 'best', 'kickboxer', 'in', 'the', 'world', ',', 'however', 'he', 'has', 'been', 'involved', 'in', 'a', 'number', 'of', 'controversies', 'relating', 'to', 'his', '"', 'unsportsmanlike', 'conducts', '"', 'in', 'the', 'sp

['die', 'rhöner', 'säuwäntzt', 'are', 'a', 'skiffle', '-', 'bluesband', 'from', 'eichenzell', '-', 'lütter', 'in', 'hessen', ',', 'germany', '.', '<sep>', ' ', 'originating', 'as', 'a', 'term', 'in', 'the', 'united', 'states', 'in', 'the', 'first', 'half', 'of', 'the', '20th', 'century', ',', 'it', 'became', 'popular', 'again', 'in', 'the', 'uk', 'in', 'the', '1950s', ',', 'where', 'it', 'was', 'associated', 'with', 'artists', 'such', 'as', 'lonnie', 'donegan', ',', 'the', 'vipers', 'skiffle', 'group', ',', 'ken', 'colyer', 'and', 'chas', 'mcdevitt', '.', '<sep>']
[['die', 'rhöner', 'säuwäntzt', 'are', 'a', 'skiffle', '-', 'bluesband', 'from', 'eichenzell', '-', 'lütter', 'in', 'hessen', ',', 'germany', '.'], [' ', 'originating', 'as', 'a', 'term', 'in', 'the', 'united', 'states', 'in', 'the', 'first', 'half', 'of', 'the', '20th', 'century', ',', 'it', 'became', 'popular', 'again', 'in', 'the', 'uk', 'in', 'the', '1950s', ',', 'where', 'it', 'was', 'associated', 'with', 'artists', 'suc

[[' ', 'her', 'self', '-', 'titled', 'debut', 'studio', 'album', 'was', 'released', 'on', '2', 'june', '2017', '.'], ['"', 'new', 'rules', '"', 'is', 'a', 'song', 'by', 'english', 'singer', 'dua', 'lipa', 'from', 'her', 'eponymous', 'debut', 'studio', 'album', '(', '2017', ')', '.']]

['joseph', 'john', '“', 'joe', '”', 'heck', '(', 'born', 'october', '30', ',', '1961', ')', 'is', 'an', 'american', 'politician', ',', 'physician', ',', 'and', 'u.s', '.', 'army', 'brigadier', 'general', 'who', 'had', 'served', 'as', 'the', 'u.s', '.', 'representative', 'for', 'nevada', "'s", '3rd', 'congressional', 'district', 'from', '2011', 'to', '2017', '.', '<sep>', ' ', 'he', 'ran', 'unsuccessfully', 'against', 'democrat', 'catherine', 'cortez', 'masto', 'in', 'the', 'general', 'election', 'for', 'the', 'open', 'nevada', 'united', 'states', 'senate', 'seat', 'in', '2016', '.', '<sep>', ' ', 'she', 'previously', 'served', 'as', 'the', '32nd', 'attorney', 'general', 'of', 'nevada', 'from', '2007', 'to

[['the', 'quality', 'cafe', '(', 'also', 'known', 'as', 'quality', 'diner', ')', 'is', 'a', 'now', '-', 'defunct', 'diner', 'at', '1236', 'west', '7th', 'street', 'in', 'los', 'angeles', ',', 'california', '.'], [' ', 'the', 'restaurant', 'ceased', 'to', 'function', 'as', 'a', 'diner', 'in', 'late', '2006', 'but', 'has', 'appeared', 'as', 'a', 'location', 'featured', 'in', 'a', 'number', 'of', 'hollywood', 'films', ',', 'including', '"', 'training', 'day', '"', ',', '"', 'old', 'school', '"', ',', '"', 'se7en', '"', ',', '"', 'ghost', 'world', '"', ',', '"', 'gone', 'in', '60', 'seconds', '"', ',', '"', 'the', 'stepfather', '"', ',', '"', 'what', "'s", 'love', 'got', 'to', 'do', 'with', 'it', '"', ',', '"', 'sex', 'and', 'death', '101', '"', ',', 'and', '"', 'catch', 'me', 'if', 'you', 'can', '.', '"'], ['old', 'school', 'is', 'a', '2003', 'american', 'comedy', 'film', 'released', 'by', 'dreamworks', 'pictures', 'and', 'the', 'montecito', 'picture', 'company', 'and', 'directed', 'by', 

...

[['gigantic', 'is', 'a', '2008', 'independent', 'comedy', 'film', 'directed', 'by', 'matt', 'aselton', 'and', 'starring', 'paul', 'dano', ',', 'zooey', 'deschanel', ',', 'john', 'goodman', ',', 'edward', 'asner', 'and', 'jane', 'alexander', '.'], [' ', 'early', 'in', 'his', 'career', ',', 'he', 'was', 'best', 'known', 'for', 'playing', 'dan', 'conner', 'on', 'the', 'abc', 'tv', 'series', '"', 'roseanne', '"', '(', '1988–1997', ')', ',', 'for', 'which', 'he', 'won', 'a', 'golden', 'globe', 'award', 'for', 'best', 'actor', 'in', '1993', '.']]

[' ', 'gabby', 'logan', 'and', 'vernon', 'kay', 'present', 'the', 'show', ',', 'whilst', 'team', 'gb', 'olympic', 'bronze', 'medal', 'winning', 'diver', 'tom', 'daley', 'is', 'the', 'expert', 'mentor', 'to', 'the', 'celebrities', '.', '<sep>', ' ', 'tom', 'daley', 'specialises', 'in', 'the', '10-metre', 'platform', 'event', 'and', 'is', 'a', 'double', 'world', 'champion', 'in', 'the', 'event', ';', 'he', 'won', 'the', '2009', 'fina', 'world', 'cham

[[' ', 'dysteleology', 'is', 'an', 'aggressive', ',', 'yet', 'optimistic', ',', 'form', 'of', 'science', '-', 'oriented', 'atheism', 'originally', 'perhaps', 'associated', 'with', 'haeckel', 'and', 'his', 'followers', ',', 'but', 'now', 'perhaps', 'more', 'associated', 'with', 'the', 'type', 'of', 'atheism', 'of', 'richard', 'dawkins', ',', 'sam', 'harris', ',', 'or', 'christopher', 'hitchens', '.'], [' ', 'he', 'contributed', 'to', '"', 'new', 'statesman', '"', ',', '"', 'the', 'nation', '"', ',', '"', 'the', 'weekly', 'standard', '"', ',', '"', 'the', 'atlantic', '"', ',', '"', 'london', 'review', 'of', 'books', '"', ',', '"', 'the', 'times', 'literary', 'supplement', '"', ',', '"', 'slate', '"', ',', '"', 'free', 'inquiry', '"', 'and', '"', 'vanity', 'fair', '"', '.']]

['r', 'adams', 'cowley', '(', 'july', '25', ',', '1917', '–', 'october', '27', ',', '1991', ')', ',', 'the', '"', 'father', 'of', 'trauma', 'medicine', ',', '"', 'was', 'an', 'american', 'surgeon', 'considered', 'a',

['kim', 'yool', '-', 'ho', '(;', 'born', 'february', '21', ',', '1985', ')', 'is', 'a', 'south', 'korean', 'actor', 'and', 'model', ',', 'he', 'is', 'best', 'known', 'for', 'his', 'role', 'in', '"', 'the', 'map', 'against', 'the', 'world', '"', ',', 'in', 'which', 'he', 'played', 'prince', 'geumwi', ',', 'and', 'the', '2016', 'zombie', 'thriller', 'apocalypse', 'film', '"', 'train', 'to', 'busan', '"', 'as', '"', 'man', 'in', 'suit', '2', '"', ',', 'who', 'helps', 'yong', '-', 'suk', '(', 'played', 'by', 'kim', 'eui', '-', 'sung', ')', '.', '<sep>', 'train', 'to', 'busan', '(', ')', 'is', 'a', '2016', 'south', 'korean', 'zombie', 'apocalypse', 'action', 'thriller', 'film', 'directed', 'by', 'yeon', 'sang', '-', 'ho', 'and', 'starring', 'gong', 'yoo', ',', 'jung', 'yu', '-', 'mi', ',', 'and', 'ma', 'dong', '-', 'seok', '.', '<sep>']
[['kim', 'yool', '-', 'ho', '(;', 'born', 'february', '21', ',', '1985', ')', 'is', 'a', 'south', 'korean', 'actor', 'and', 'model', ',', 'he', 'is', 'best'

[' ', 'he', 'currently', 'serves', 'as', 'ceo', 'of', 'lionsgate', 'uk', '&', 'europe', 'and', 'was', 'the', 'executive', 'producer', 'for', 'brooklyn', ',', 'eddie', 'the', 'eagle', ',', 'salmon', 'fishing', 'in', 'the', 'yemen', ',', 'the', 'railway', 'man', 'and', 'london', 'has', 'fallen', '.', '<sep>', 'london', 'has', 'fallen', 'is', 'a', '2016', 'american', 'action', 'thriller', 'film', 'directed', 'by', 'babak', 'najafi', 'and', 'written', 'by', 'creighton', 'rothenberger', ',', 'katrin', 'benedikt', ',', 'chad', 'st', '.', 'john', 'and', 'christian', 'gudegast', '.', '<sep>']
[[' ', 'he', 'currently', 'serves', 'as', 'ceo', 'of', 'lionsgate', 'uk', '&', 'europe', 'and', 'was', 'the', 'executive', 'producer', 'for', 'brooklyn', ',', 'eddie', 'the', 'eagle', ',', 'salmon', 'fishing', 'in', 'the', 'yemen', ',', 'the', 'railway', 'man', 'and', 'london', 'has', 'fallen', '.'], ['london', 'has', 'fallen', 'is', 'a', '2016', 'american', 'action', 'thriller', 'film', 'directed', 'by',

[['children', '’s', 'national', 'medical', 'center', '(', 'formerly', 'dc', 'children', '’s', 'hospital', ')', 'is', 'ranked', 'among', 'the', 'top', '10', 'children', '’s', 'hospitals', 'in', 'the', 'country', 'by', '"', 'u.s', '.', 'news', '&', 'world', 'report', '.', '"'], ['medstar', 'washington', 'hospital', 'center', 'is', 'the', 'largest', 'private', 'hospital', 'in', 'washington', ',', 'd.c', '.', 'a', 'member', 'of', 'medstar', 'health', ',', 'the', 'not', '-', 'for', '-', 'profit', 'hospital', 'center', 'is', 'licensed', 'for', '926', 'beds', '.']]



In [5]:
mydata.train.examples[0].Question

[[[]]]

In [5]:
x = torch.randn(2,10,300)

#(self,word_dim,n_head,n_hid,dropout,nlayers):
model = CSAEncoder(word_dim=300,n_head=4,n_hid=300,dropout=0.1,nlayers=2)
print(model(x).shape)


torch.Size([2, 300])


# trans data

In [2]:
src = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/train.json'
des = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/small_train_sep_100.csv'

dataProcesser(src,des,100)
src = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/dev.json'
des = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/small_dev_sep_100.csv'
dataProcesser(src,des,100)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


write down


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


write down


[4, 5, 6, 4]


In [12]:
s = "time is out (so) i will kill you.Am i a joke?"

s = s.replace('(',' ( ')

print(s.split(' '))

['time', 'is', 'out', '', '(', 'so)', 'i', 'will', 'kill', 'you.Am', 'i', 'a', 'joke?']
